In [31]:
from langgraph.graph import START, StateGraph, END
from IPython.display import Image, display
from typing import Any, TypedDict, Annotated
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from operator import add
from models import get_model

class State(TypedDict):
    question: str
    answer: str
    context: Annotated[list, add]

def search_web(state: State):
    '''Retrive doc from web search'''

    # Search
    tavily_search = TavilySearchResults(max_results=3)
    search_docs = tavily_search.invoke(state['question'])

    print(search_docs)

    #formate
    formatted_docs = "\n\n---\n\n".join([
        f'<Document href="{doc.get("url", "")}">\n{doc.get("content", "")}\n</Document>' for doc in search_docs
    ])

    print('formatted_docs --> ', formatted_docs)
    return {"context": [formatted_docs]}

def search_wikipedia(state: State):
    '''Retrive doc from Wikipedia'''

    # Search
    loader = WikipediaLoader(query=state['question'], load_max_docs=5)
    docs = loader.load()

    # Format
    formatted_docs = "\n\n---\n\n".join([
        f'<Document source="{doc.metadata["source"]}" page="{doc.metadata.get("page", "")}">\n{doc.page_content}\n</Document>' for doc in docs
    ])

    print('formatted_docs wiki--> ', formatted_docs)

    return {"context": [formatted_docs]}

def generate_answer(state: State):
    '''Generate answer using LLM'''

    #get Context
    context = state['context']
    question = state['question']

    #Template
    answer_template = '''Answer the question {question} using this context:{context}'''
    answer_instructions = answer_template.format(question=question, context=context)
    
    # Get model
    model = get_model('azure')

    answer = model.invoke([SystemMessage(content=answer_instructions)] + [HumanMessage(content=f"Answer the question.")]),
    
    return {"answer": answer}

In [32]:
graph_builder = StateGraph(State)

graph_builder.add_node('search_web', search_web)
graph_builder.add_node('search_wikipedia', search_wikipedia)
graph_builder.add_node('generate_answer', generate_answer)

graph_builder.add_edge(START, 'search_web')
graph_builder.add_edge(START, 'search_wikipedia')
graph_builder.add_edge(['search_web', 'search_wikipedia'], 'generate_answer')
graph_builder.add_edge('generate_answer', END)

react_graph = graph_builder.compile()
# display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [33]:
result = react_graph.invoke({"question": "What are expectations for Nvidia q2 2024 earning."})
print(result)

ProxyError(MaxRetryError('HTTPSConnectionPool(host=\'api.tavily.com\', port=443): Max retries exceeded with url: /search (Caused by ProxyError(\'Unable to connect to proxy\', NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x124f13510>: Failed to resolve \'genproxy\' ([Errno 8] nodename nor servname provided, or not known)")))'))


AttributeError: 'str' object has no attribute 'get'